In [3]:
pip install flask flask-bcrypt flask-login dash dash-bootstrap-components plotly pandas


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from flask_bcrypt import Bcrypt

# Initialize bcrypt
bcrypt = Bcrypt()

# Read your business.csv file
business_df = pd.read_csv("/Users/shashankmidididdi/Downloads/business.csv")

# Define a default password for all users
default_password = "demo1234"
password_hash = bcrypt.generate_password_hash(default_password).decode("utf-8")

# Create user entries for each business
user_access = []
for i, row in business_df.iterrows():
    business_id = row["business_id"]
    name = row["name"]
    username = name.lower().replace(" ", "_") + "@demo.com"  # e.g., "st_honore_pastries@demo.com"
    
    user_access.append({
        "business_id": business_id,
        "username": username,
        "password_hash": password_hash,
        "role": "user"
    })

# Add an admin account
user_access.append({
    "business_id": "*",
    "username": "admin@demo.com",
    "password_hash": password_hash,
    "role": "admin"
})

# Save to CSV
user_access_df = pd.DataFrame(user_access)
user_access_df.to_csv("user_access.csv", index=False)

print("✅ user_access.csv created successfully with", len(user_access_df), "users")
print(user_access_df.head())


✅ user_access.csv created successfully with 350 users
              business_id                        username  \
0  MTSW4McQd7CbVtyjqoe9mw     st_honore_pastries@demo.com   
1  k0hlBqXX-Bt0vf1op7Jr1w  tsevi's_pub_and_grill@demo.com   
2  il_Ro8jwPlHresjw9EGmBg                denny's@demo.com   
3  0bPLkL0QhhPO5kt1_EXmNQ   zio's_italian_market@demo.com   
4  MUTTqe8uqyMdBl186RmNeA               tuna_bar@demo.com   

                                       password_hash  role  
0  $2b$12$ZZSaIQdlRp3R9sR1BDD1yO0QclBjrnIdG.QD.F....  user  
1  $2b$12$ZZSaIQdlRp3R9sR1BDD1yO0QclBjrnIdG.QD.F....  user  
2  $2b$12$ZZSaIQdlRp3R9sR1BDD1yO0QclBjrnIdG.QD.F....  user  
3  $2b$12$ZZSaIQdlRp3R9sR1BDD1yO0QclBjrnIdG.QD.F....  user  
4  $2b$12$ZZSaIQdlRp3R9sR1BDD1yO0QclBjrnIdG.QD.F....  user  


In [49]:
import pandas as pd
reviews_df = pd.read_csv("/Users/shashankmidididdi/Downloads/reviews.csv")
print(reviews_df)

                    review_id                 user_id             business_id  \
0      BXQcBN0iAi1lAUxibGLFzA  6_SpY41LIHZuIaiDs5FMKA  MTSW4McQd7CbVtyjqoe9mw   
1      uduvUCvi9w3T2bSGivCfXg  tCXElwhzekJEH6QJe3xs7Q  MTSW4McQd7CbVtyjqoe9mw   
2      a0vwPOqDXXZuJkbBW2356g  WqfKtI-aGMmvbA9pPUxNQQ  MTSW4McQd7CbVtyjqoe9mw   
3      MKNp_CdR2k2202-c8GN5Dw  3-1va0IQfK-9tUMzfHWfTA  MTSW4McQd7CbVtyjqoe9mw   
4      D1GisLDPe84Rrk_R4X2brQ  EouCKoDfzaVG0klEgdDvCQ  MTSW4McQd7CbVtyjqoe9mw   
...                       ...                     ...                     ...   
39299  TrDOGEiPLwTFbfr29faHkw  rnZ9TEFIRjfyKQwmpizG_w  CjkxTRergWcYWb4SVW3gxw   
39300  vAwfNxH2wuJ6ijKF_E5wiA  z1z8fZ-NS1uomgoRkuFOpw  CjkxTRergWcYWb4SVW3gxw   
39301  UI9PQwq9aB1CoQ4BRkxXQA  erkAc2nslUWHpnJpA9SfUg  CjkxTRergWcYWb4SVW3gxw   
39302  aTXYqv5m98iWfeOT33nndA  QuT78K-kZIXML-Qx2_5-Hg  CjkxTRergWcYWb4SVW3gxw   
39303  Ic0cGXrlrkAsO4vgHiCtWA  OdVXRQ2bLb9vNJhHkL9dgg  CjkxTRergWcYWb4SVW3gxw   

       stars  useful  funny

In [43]:
import pandas as pd
reviews_df = pd.read_csv("/Users/shashankmidididdi/Downloads/reviews.csv")
print(reviews_df.columns.tolist())


['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']


In [45]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

reviews_df["sentiment_compound"] = reviews_df["text"].astype(str).apply(lambda x: sia.polarity_scores(x)["compound"])

def label(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

reviews_df["sentiment_label"] = reviews_df["sentiment_compound"].apply(label)

# Save this enriched version
reviews_df.to_csv("reviews_with_sentiment.csv", index=False)
print("✅ Saved enriched reviews file with sentiment columns!")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/shashankmidididdi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


✅ Saved enriched reviews file with sentiment columns!


In [53]:
import pandas as pd
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
import nltk
nltk.download('vader_lexicon')

# Load both datasets
business_df = pd.read_csv("/Users/shashankmidididdi/Downloads/business.csv")
reviews_df = pd.read_csv("/Users/shashankmidididdi/Downloads/reviews.csv")

# Perform sentiment analysis
sia = SentimentIntensityAnalyzer()
reviews_df["sentiment_compound"] = reviews_df["text"].astype(str).apply(lambda x: sia.polarity_scores(x)["compound"])

def label(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

reviews_df["sentiment_label"] = reviews_df["sentiment_compound"].apply(label)

# Merge with business info
merged_df = pd.merge(reviews_df, business_df, on="business_id", how="left")

# Extract first category
merged_df["category_main"] = merged_df["categories"].astype(str).apply(lambda x: x.split(",")[0].strip() if pd.notna(x) else "Unknown")

# Aggregation 1 – Daily sentiment trend
merged_df["date"] = pd.to_datetime(merged_df["date"], errors="coerce")
valid = merged_df.dropna(subset=["date"])
agg_sentiment_daily = (
    valid.groupby(["business_id", valid["date"].dt.date])
    .agg(avg_sentiment=("sentiment_compound", "mean"),
         avg_stars=("stars_x", "mean") if "stars_x" in valid.columns else ("stars", "mean"),
         review_count=("review_id", "count"))
    .reset_index()
)
agg_sentiment_daily.to_csv("agg_sentiment_daily.csv", index=False)

# Aggregation 2 – Category sentiment
agg_category_sentiment = (
    merged_df.groupby("category_main")
    .agg(avg_sentiment=("sentiment_compound", "mean"),
         avg_stars=("stars_x", "mean") if "stars_x" in merged_df.columns else ("stars", "mean"),
         review_count=("review_id", "count"))
    .reset_index()
)
agg_category_sentiment.to_csv("agg_category_sentiment.csv", index=False)

# Aggregation 3 – Top words by category
# Aggregation 3 – Top words by category (cleaned version)
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import re

# Download resources for stopword removal & lemmatization
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# Optional: remove generic restaurant terms that appear too often
custom_exclude = {
    "restaurant", "food", "place", "menu", "time",
    "people", "service", "staff", "order", "table", "location"
}

def extract_keywords(text):
    """Extract clean, meaningful keywords from review text."""
    words = re.findall(r"\b[a-zA-Z]{3,}\b", str(text).lower())
    filtered = [
        lemmatizer.lemmatize(w)
        for w in words
        if w not in stop_words and w not in custom_exclude
    ]
    return filtered

# Apply to each review
merged_df["keywords"] = merged_df["text"].apply(extract_keywords)

# Aggregate top words by category
records = []
for cat, subset in merged_df.groupby("category_main"):
    all_words = [w for words in subset["keywords"] for w in words]
    top10 = Counter(all_words).most_common(10)
    for word, count in top10:
        records.append({"category": cat, "keyword": word, "count": count})

agg_top_words_by_category = pd.DataFrame(records)
agg_top_words_by_category.to_csv("agg_top_words_by_category.csv", index=False)

print("✅ Cleaned & meaningful top words CSV created successfully!")


print("✅ All 3 Tableau-ready CSVs created successfully with real sentiment data!")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/shashankmidididdi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shashankmidididdi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shashankmidididdi/nltk_data...


✅ Cleaned & meaningful top words CSV created successfully!
✅ All 3 Tableau-ready CSVs created successfully with real sentiment data!


In [27]:
business_df = pd.read_csv("/Users/shashankmidididdi/Downloads/business.csv")
reviews_df = pd.read_csv("/Users/shashankmidididdi/Downloads/reviews.csv")
daily_df = pd.read_csv("agg_sentiment_daily.csv")
category_df = pd.read_csv("agg_category_sentiment.csv")
topwords_df = pd.read_csv("agg_top_words_by_category.csv")


In [29]:
!pip install dash plotly


In [55]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# 1️⃣ Load your processed data
reviews_df = pd.read_csv("/Users/shashankmidididdi/Downloads/Applied_Project/reviews_with_sentiment.csv")
business_df = pd.read_csv("/Users/shashankmidididdi/Downloads/business.csv")
daily_df = pd.read_csv("/Users/shashankmidididdi/Downloads/Applied_Project/agg_sentiment_daily.csv")
topwords_df = pd.read_csv("/Users/shashankmidididdi/Downloads/Applied_Project/agg_top_words_by_category.csv")

# 2️⃣ Merge reviews with business info for dropdown names
merged = pd.merge(reviews_df, business_df[["business_id", "name", "categories"]], on="business_id", how="left")

# Ensure date is parsed
daily_df["date"] = pd.to_datetime(daily_df["date"], errors="coerce")

# 3️⃣ Create dropdown options
business_options = [{"label": name, "value": bid} for bid, name in zip(business_df["business_id"], business_df["name"])]

# 4️⃣ Initialize Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("🍽️ Restaurant Sentiment Dashboard", style={"textAlign": "center"}),

    html.Div([
        html.Label("Select a Restaurant:"),
        dcc.Dropdown(
            id="business_dropdown",
            options=business_options,
            value=business_df["business_id"].iloc[0],
            style={"width": "60%"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div([
        dcc.Graph(id="sentiment_trend"),
        dcc.Graph(id="sentiment_pie"),
        dcc.Graph(id="top_words_bar")
    ])
])

# 5️⃣ Define interactivity
@app.callback(
    [Output("sentiment_trend", "figure"),
     Output("sentiment_pie", "figure"),
     Output("top_words_bar", "figure")],
    [Input("business_dropdown", "value")]
)
def update_dashboard(selected_business_id):
    # Filter daily trend
    df_trend = daily_df[daily_df["business_id"] == selected_business_id]

    if df_trend.empty:
        fig_empty = px.scatter(title="⚠️ No data available for this restaurant")
        return fig_empty, fig_empty, fig_empty

    # Filter reviews for sentiment distribution
    df_reviews = reviews_df[reviews_df["business_id"] == selected_business_id]

    # Identify category for top words
    business_row = business_df[business_df["business_id"] == selected_business_id]
    category_name = ""
    if not business_row.empty:
        cats = str(business_row["categories"].iloc[0])
        category_name = cats.split(",")[0].strip() if cats else "Unknown"
    df_words = topwords_df[topwords_df["category"] == category_name]

    # --- Chart 1: Daily Sentiment Trend (Avg Sentiment & Stars) ---
    fig_trend = px.line(df_trend, x="date", y="avg_sentiment",
                        title=f"Average Sentiment Over Time for {business_row['name'].iloc[0]}",
                        markers=True)
    fig_trend.add_scatter(x=df_trend["date"], y=df_trend["avg_stars"],
                          mode="lines+markers", name="Average Stars", line=dict(color="orange"))
    fig_trend.update_layout(xaxis_title="Date", yaxis_title="Score (Sentiment & Stars)")

    # --- Chart 2: Sentiment Distribution Pie ---
    if "sentiment_label" in df_reviews.columns:
        sentiment_counts = df_reviews["sentiment_label"].value_counts().reset_index()
        sentiment_counts.columns = ["sentiment", "count"]
        fig_pie = px.pie(sentiment_counts, names="sentiment", values="count",
                         color="sentiment",
                         color_discrete_map={"Positive": "green", "Neutral": "gray", "Negative": "red"},
                         title="Sentiment Distribution")
    else:
        fig_pie = px.scatter(title="⚠️ Sentiment label not found in reviews data")

    # --- Chart 3: Top Words by Category ---
    if not df_words.empty:
        fig_bar = px.bar(df_words.sort_values(by="count", ascending=False),
                         x="keyword", y="count",
                         title=f"Top Keywords in Category ({category_name})",
                         text_auto=True)
    else:
        fig_bar = px.scatter(title=f"⚠️ No top words found for category '{category_name}'")

    return fig_trend, fig_pie, fig_bar


# 6️⃣ Run the app
app.run(debug=False, use_reloader=False)

